# Borrador Análisis exploratorio e datos

**Enunciado**

*La idea de esta sección es que exploren el dataset y obtengan una idea de como son los datos de su problema para que en la siguientes secciones, puedan modelarlos.*

- [x] Analizar cantidad de datos nulos, tipos de datos, duplicados
- [ ] Distribuciones de las variables a través de histogramas.
- [ ] Generar visualizaciones de las interacciones (como por ejemplo, una scatter matrix) en las distintas variables.
- [ ] Ver las correlaciones entre las distintas variables y los valores faltantes de cada una de estas. 
- [ ] Proyectar los datos con PCA, T-SNE o UMAP para ver si existen relaciones entre las distintas variables de forma bi o tri dimensional.
- [ ] Reportar los patrones y relaciones interesantes.
- [ ] MÁS (No restringirse a lo mencionado anteriormente).

**Ejemplo enunciado**

Al efectuar el análisis exploratorio de datos pudimos notar lo siguiente:

El atributo A tenía X datos nulos y Y duplicados... que podrían potencialmente dificultar el análisis.

La variable A se distribuye de forma normal. En cambio la B en forma exponencial. La variable C es categórica.

Los atributos A y B interactúan de P forma, mientras que los C y D de Q forma...

Los atributos A y C se correlacionan con las variables de salida Y de forma creciente... B no tiene correlación con la salida.

Además, encontramos que al proyectar en 2D ...

In [1]:
import pandas as pd
import numpy as np
# ...

In [2]:
df_train = pd.read_pickle('train.pickle')

## Análisis Básico

In [3]:
def exploratory_data_analysis(dataframe):
    print(f'El DataFrame tiene {dataframe.shape[0]} filas y {dataframe.shape[1]} columnas')
    print(f'El DataFrame esta compuesto por las siguientes columnas: {dataframe.columns.to_list()}')
    # print(f'\n3.- Ejemplos de filas del DataFrame:')
    print('Descripción numérica del DataFrame:')
    display(dataframe.describe())
    print('Cantidad de valores nulos por columna:')
    display(dataframe.isna().sum())
    print('Cantidad de valores únicos por columna:')
    display(dataframe.nunique())

exploratory_data_analysis(df_train)

El DataFrame tiene 7881 filas y 16 columnas
El DataFrame esta compuesto por las siguientes columnas: ['name', 'release_date', 'english', 'developer', 'publisher', 'platforms', 'required_age', 'categories', 'genres', 'tags', 'achievements', 'average_playtime', 'price', 'short_description', 'estimated_sells', 'rating']
Descripción numérica del DataFrame:


,english,required_age,achievements,average_playtime,price,estimated_sells
count,7881.000000,7881.00000,7881.000000,7881.000000,7881.000000,7.881000e+03
mean,0.985789,0.78924,43.170156,439.296790,8.431342,2.105767e+05
std,0.118369,3.55538,265.399206,3303.162083,8.755668,1.513926e+06
min,0.000000,0.00000,0.000000,0.000000,0.000000,3.600000e+03
25%,1.000000,0.00000,0.000000,0.000000,1.990000,9.724000e+03
50%,1.000000,0.00000,15.000000,27.000000,6.990000,2.150800e+04
75%,1.000000,0.00000,35.000000,251.000000,11.390000,7.357300e+04
max,1.000000,18.00000,9821.000000,190625.000000,78.990000,7.944129e+07


Cantidad de valores nulos por columna:


name                 0
release_date         0
english              0
developer            0
publisher            0
platforms            0
required_age         0
categories           0
genres               0
tags                 0
achievements         0
average_playtime     0
price                0
short_description    0
estimated_sells      0
rating               0
dtype: int64

Cantidad de valores únicos por columna:


name                 7881
release_date         2251
english                 2
developer            5365
publisher            3992
platforms               5
required_age            6
categories           1933
genres                844
tags                 3981
achievements          280
average_playtime     1257
price                 162
short_description    7848
estimated_sells      4879
rating                  5
dtype: int64

### Columnas con varias categorías posibles

In [6]:
def categorias(df,columna):
    separate = lambda string: np.array(string.split(';'),dtype=object)
    options = np.vectorize(separate)(df[columna].unique())
    return np.unique(np.concatenate(options))

print("---\nplataforms")
print(categorias(df_train,'platforms'))

print('---\ncategories')
print(categorias(df_train,'categories'))

print('---\ngenres')
print(categorias(df_train,'genres'))

print('---\ntags (total de tags disponibles)')
print(len(categorias(df_train,'tags')))

---
plataforms
['linux' 'mac' 'windows']
---
categories
['Captions available' 'Co-op' 'Commentary available'
 'Cross-Platform Multiplayer' 'Full controller support' 'In-App Purchases'
 'Includes Source SDK' 'Includes level editor' 'Local Co-op'
 'Local Multi-Player' 'MMO' 'Mods' 'Mods (require HL2)' 'Multi-player'
 'Online Co-op' 'Online Multi-Player' 'Partial Controller Support'
 'Shared/Split Screen' 'Single-player' 'Stats' 'Steam Achievements'
 'Steam Cloud' 'Steam Leaderboards' 'Steam Trading Cards'
 'Steam Turn Notifications' 'Steam Workshop' 'SteamVR Collectibles'
 'VR Support' 'Valve Anti-Cheat enabled']
---
genres
['Action' 'Adventure' 'Animation & Modeling' 'Audio Production' 'Casual'
 'Design & Illustration' 'Early Access' 'Education' 'Free to Play'
 'Game Development' 'Gore' 'Indie' 'Massively Multiplayer' 'Nudity'
 'Photo Editing' 'RPG' 'Racing' 'Sexual Content' 'Simulation'
 'Software Training' 'Sports' 'Strategy' 'Utilities' 'Video Production'
 'Violent' 'Web Publishing']

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class Categories(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.algo = None
        return self

    def transform(self, X):
        return None

## Descripciones de las variables a través de histogramas